<a href="https://colab.research.google.com/github/Ruchir1218/Portfolio_Projects/blob/main/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pyspark questions practice

In [1]:
# Read a file and count word frequency in that file

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Read the file
df = spark.read.text("data/sample.txt")

# Split the text into words
words = df.select(explode(split(col("value"), " ")).alias("word"))